## ABC

In [1]:
import sys 
from pathlib import Path

# Define the path to the src directory
src_path = Path().resolve().parent / 'src'

# Add src to the Python path
if src_path not in sys.path:
    sys.path.append(str(src_path))

# Check if src is in sys.path
#print(f"sys.path: {sys.path}")

In [2]:
import numpy as np
from simulators import prior, simulator, simulator2
from models.abc import abc_inference

In [6]:
# Define the dimensions and the number of samples
N = 100000
dim = 2 # 2, 5, 10

# Observation
y_obs = np.array(0.3)
# Use seed to have the same value for x in each run
np.random.seed(42)
x = np.random.uniform(-0.5, 0.5, size = (dim,))
# threshold
e = 0.01

# Sampling thetas
np.random.seed(None)
theta_pr = prior(N, dim)

# Select the simulator
selected_sim = simulator2

# List of the accepted samples
samples_pos = abc_inference(y_obs, x, theta_pr, e, selected_sim)

In [7]:
samples_pos.shape #simulator: 2:1082 , 5:947 , 10:678, 20:493, simulator2: 2: 1084, 5:966, 10: 665, 20: 489

(1098, 2)

In [8]:
samples_pos[:2, :]

array([[0.7983313, 1.0521845],
       [0.5350693, 0.6302228]], dtype=float32)

## OMC

In [2]:
import numpy as np
from simulators import prior, simulator_omc
from models.omc import OMCInference

In [4]:
# Define the dimensions and number of samples
N=10000
theta_dim, u_dim = 5, 5 # Dimensions of theta, # Dimension of nuisance variables

# Observation
y_obs = np.full(theta_dim, 0.3) # Assuming y has the same dimension as simulator output
np.random.seed(42)

# Fixed x from a uniform distribution
x_fixed = np.random.uniform(-1,1, theta_dim)

# Sampling thetas from the prior
theta_pr = prior(N, theta_dim)

# Run Inference 
omc_inference = OMCInference(theta_pr, y_obs, x_fixed, u_dim)
samples_pos, weights = omc_inference.infer()
print(f"The number of samples is: {samples_pos.shape[0]}")

# Calculate weighted mean and std for each dimnensioin of theta
weighted_mean = np.average(samples_pos, weights = weights, axis=0)
weighted_variance = np.average((samples_pos - weighted_mean)**2, weights = weights, axis=0)
weighted_std = np.sqrt(weighted_variance)

print(f"The mean value of the posterior distribution is: {weighted_mean}")
print(f"The standard deviation of the posterior distribution is: {weighted_std}")

The number of samples is: 10000
The mean value of the posterior distribution is: [ 0.00538229 -0.02723251 -0.00138771  0.01190336 -0.01144511]
The standard deviation of the posterior distribution is: [1.15842172 1.15358599 1.15297844 1.15212796 1.15044569]


In [5]:
weights.shape

(10000,)

In [7]:
y_obs

array([0.3, 0.3, 0.3, 0.3, 0.3])

In [9]:
theta_pr.shape

(10000, 5)

In [10]:
non_zero_mask = weights>0
filtered_samples = samples_pos[non_zero_mask]
filtered_weights = weights[non_zero_mask]
print(f"The number of the non-negative samples are: {filtered_samples.shape}")

The number of the non-negative samples are: (10000, 5)
